In [ ]:
import pathlib
import pandas as pd
import numpy as np
import tensorflow as tf
from numpy.random import seed
from keras import layers
from keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# read_csv加了个header=None，会多读取一行。后续训练要修正一下input shape
### 运行一次后自动加载
### 1. 两个分类的音频csv自动加载
### 2. 合并channel形成4d矩阵后，生成对应label
### 3. 构建model，训练model
### 4. 存储val_loss最小的model
### 5. 初步以0.1为跨度检查哪个区间的阈值，f1 score最高
### 6. 在f1 score最高的区间内，再细分1000个刻度进一步检查最合适阈值。
### 7. 输出结果
### 可选（调用matlab将两个class的音频文件转换为对应矩阵）

In [ ]:
def readCsv(fileName):
    filePath = f'/content/drive/MyDrive/0819/{fileName}'
    folder = pathlib.Path.cwd().parent.joinpath(filePath)
    print(folder)
    soundMap = dict()  # map[string]list, list elems are np array

    for fp in folder.iterdir():
        if fp.match('*.csv'):
            # 因为有可能名字里带'.'，不能split('.'后取index0. 那就划分完把整个list都取出来，去掉最后index[-1]的csv部分做个拼接)
            varname = fp.parts[-1].split('.')
            # print(type(varname))
            # print(varname)
            # if "ScountRSound_1_7764_0_20220801173146669-3" in varname[0]:
            #   print(type(varname))
            #   print(varname)
            soundMatrix = pd.read_csv(fp,header=None).values
            # print(varname[:-8])  # each file has 4 matrices: Log, Org, Amp, Phs

            # ignore the matrices type of sound matrix
            varnameList = ','.join(varname[:-1])
            # print(varnameList)
            soundFileName = varnameList[:-8]
            # print(soundFileName)
            if soundFileName not in soundMap:
                soundMap[soundFileName] = []  
            soundMap[soundFileName].append(soundMatrix)
            # print("type is ", type(soundMatrix))
            # print(soundMatrix.shape)
    for key in soundMap:
      if len(soundMap[key])!=4:
        print(key)
    return soundMap
# readCsv('normal')

### get normal and error sound matrix

In [ ]:
normalSound = readCsv('normal')
# normalSound['ScountRSound_1_7764_0_20220801173146669-3'].append(pd.read_csv("/content/drive/MyDrive/0819/normal/ScountRSound_1_7764_0_20220801173146669-3STFT_Phs.csv"))
normalSoundList = []
for v in normalSound.values():
    if np.array(v).shape!= (4,38,30):
      print(np.array(v).shape)
    normalSoundList.append(np.array(v))
normalSoundList = np.array(normalSoundList)
print(normalSoundList.shape)
transposeNormalSoundList = normalSoundList.transpose((0, 2, 3, 1))
# transpose of normal sound
print(transposeNormalSoundList.shape)

/content/drive/MyDrive/0819/normal
(630, 4, 38, 30)
(630, 38, 30, 4)


In [ ]:
errorSound = readCsv('error')
errSoundList = []
for v in errorSound.values():
    # print(np.array(v).shape)
    errSoundList.append(np.array(v))
errSoundList = np.array(errSoundList)
transposeErrorSoundList = errSoundList.transpose((0, 2, 3, 1))

# transpose of error sound
print(transposeErrorSoundList.shape)

/content/drive/MyDrive/0819/error
(570, 38, 30, 4)


In [ ]:
# seed(123)


### get sound labels

In [ ]:
normalLabels = np.ones(len(normalSound), dtype=int)
errorLabels = np.zeros(len(errorSound), dtype=int)
print(normalLabels.shape)

totalLabels = np.concatenate((normalLabels, errorLabels),axis=0)
print('len of labels',len(totalLabels))
print(totalLabels)

totalData = np.concatenate((transposeNormalSoundList, transposeErrorSoundList), axis=0)
print(totalData.shape)

(630,)
len of labels 1200
[1 1 1 ... 0 0 0]
(1200, 38, 30, 4)


### build model

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(totalData, totalLabels, test_size=0.2, random_state=0)
print(y_test)
# one-hot
y_train = to_categorical(y_train)
y_test= to_categorical(y_test)


# # separate train and validation set
# x, x_test, y, y_test = train_test_split(totalData, totalLabels, test_size=0.2, random_state=0)
# print(y_test)
# # one-hot
# y_train = to_categorical(y_train)
# y_test= to_categorical(y_test)
# y_test

# x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.2,random_state=0)

NameError: ignored

In [ ]:
# # from keras.constraints import MinMaxNorm
# model = models.Sequential()
# model.add(layers.Conv2D(32,
#                         (3, 3),
#                         activation='relu',
#                         padding='same',
#                         input_shape=(38, 30, 4)))
# model.add(layers.Conv2D(32,
#                         (3, 3),
#                         activation='relu',
#                         padding='same',))

# model.add(layers.Conv2D(32,
#                         (3, 3),
#                         activation='relu',
#                         padding='same',))
# model.add(layers.Conv2D(32,
#                         (3, 3),
#                         activation='relu',
#                         padding='same',))

# model.add(layers.Conv2D(32,
#                         (3, 3),
#                         activation='relu',
#                         padding='same',))
# model.add(layers.Conv2D(32,
#                         (3, 3),
#                         activation='relu',
#                         padding='same',))

# model.add(layers.Dropout(0.1))
# model.add(layers.Flatten())
# model.add(layers.Dense(32,activation='relu'))
# model.add(layers.Dense(2, activation='softmax'))


# opt = optimizers.Adam(learning_rate=0.002, decay=1e-6) #0.002， 0.0005
# model.compile(loss='categorical_crossentropy',
#               optimizer=opt,
#               metrics=['accuracy'])

# model.summary()

In [ ]:
from tensorflow import keras
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model
import datetime
import matplotlib.pyplot as plt
import numpy as np



# def residual_block(layer_input, filters):
#     """Residual block described in paper"""
#     d = Conv2D(filters, 3, activation='relu', padding='same')(layer_input)
#     d = BatchNormalization(momentum=0.8)(d)
#     d = Conv2D(filters, 3, activation='relu', padding='same')(d)
#     d = BatchNormalization(momentum=0.8)(d)
#     d = Add()([d, layer_input])
#     return d


inputs = Input(shape=(38, 30, 4))
conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
conv2 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
conv2 = Conv2D(32, 3, activation='relu', padding='same')(conv2)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(32, 3, activation='relu', padding='same')(pool1)
conv3 = Conv2D(32, 3, activation='relu', padding='same')(conv3)
conv3 = Conv2D(32, 3, activation='relu', padding='same')(conv3)
conv3 = Conv2D(32, 3, activation='relu', padding='same')(conv3)

D1 = Dropout(0.1)(conv3)
D1 = Flatten()(D1)
D1 = Dense(32,activation='relu')(D1) #relu
Out = Dense(2, activation='softmax')(D1)

model = Model(inputs, Out)
model.summary()

opt = optimizers.Adam(learning_rate=0.0001, decay=1e-6) #0.002， 0.0005
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


Model: "model_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_39 (InputLayer)       [(None, 38, 30, 4)]       0         
                                                                 
 conv2d_323 (Conv2D)         (None, 38, 30, 32)        1184      
                                                                 
 conv2d_324 (Conv2D)         (None, 38, 30, 32)        9248      
                                                                 
 conv2d_325 (Conv2D)         (None, 38, 30, 32)        9248      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 19, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_326 (Conv2D)         (None, 19, 15, 32)        9248      
                                                          

In [ ]:
x_train.shape

(960, 38, 30, 4)

In [ ]:
from keras import callbacks
callbacks_list = [callbacks.ModelCheckpoint(
    filepath = 'best_model-2.h5',
    monitor =  'val_loss',
    save_best_only = True,
    verbose = 1
)]

history = model.fit(x_train,y_train,epochs=15,batch_size=3,callbacks=callbacks_list,validation_split=0.2)


Epoch 1/15
254/256 [============================>.] - ETA: 0s - loss: 0.7113 - accuracy: 0.5446
Epoch 1: val_loss improved from inf to 0.69797, saving model to best_model-2.h5
256/256 [==============================] - 3s 7ms/step - loss: 0.7109 - accuracy: 0.5456 - val_loss: 0.6980 - val_accuracy: 0.4844
Epoch 2/15
253/256 [============================>.] - ETA: 0s - loss: 0.6641 - accuracy: 0.5968
Epoch 2: val_loss did not improve from 0.69797
256/256 [==============================] - 2s 6ms/step - loss: 0.6640 - accuracy: 0.5977 - val_loss: 0.7866 - val_accuracy: 0.4844
Epoch 3/15
252/256 [============================>.] - ETA: 0s - loss: 0.6183 - accuracy: 0.6944
Epoch 3: val_loss did not improve from 0.69797
256/256 [==============================] - 2s 6ms/step - loss: 0.6178 - accuracy: 0.6953 - val_loss: 0.7281 - val_accuracy: 0.4792
Epoch 4/15
255/256 [============================>.] - ETA: 0s - loss: 0.4877 - accuracy: 0.7895
Epoch 4: val_loss did not improve from 0.69797
25

In [ ]:
test_loss,test_acc = model.evaluate(x_test,y_test)

8/8 [==============================] - 0s 5ms/step - loss: 4.1720 - accuracy: 0.5458


In [ ]:
model.load_weights('best_model.h5')

OSError: ignored

In [ ]:
test_loss,test_acc = model.evaluate(x_test,y_test)

8/8 [==============================] - 0s 5ms/step - loss: 2.2797 - accuracy: 0.5083


In [ ]:
test_acc

0.5083333253860474

In [ ]:
test_loss

2.5064799785614014

In [ ]:
res = model.predict(x_test)

In [ ]:
res

array([[6.20374262e-01, 3.79625767e-01],
       [9.90978718e-01, 9.02128126e-03],
       [1.72427401e-01, 8.27572644e-01],
       [9.99624014e-01, 3.75939446e-04],
       [1.20573509e-06, 9.99998808e-01],
       [9.98185575e-01, 1.81439379e-03],
       [9.99928594e-01, 7.14526686e-05],
       [9.74102795e-01, 2.58972310e-02],
       [1.02784917e-01, 8.97215009e-01],
       [9.81646597e-01, 1.83533505e-02],
       [9.90113378e-01, 9.88656096e-03],
       [9.69527304e-01, 3.04727070e-02],
       [9.78133500e-01, 2.18664948e-02],
       [9.99393106e-01, 6.06844435e-04],
       [9.90590453e-01, 9.40954406e-03],
       [8.16482544e-01, 1.83517441e-01],
       [9.99807060e-01, 1.92961335e-04],
       [9.15961802e-01, 8.40381905e-02],
       [3.28693569e-01, 6.71306491e-01],
       [9.97051477e-01, 2.94846878e-03],
       [9.33169305e-01, 6.68306723e-02],
       [9.99666691e-01, 3.33289907e-04],
       [9.99920487e-01, 7.95334927e-05],
       [9.99565899e-01, 4.34136309e-04],
       [9.999501

In [ ]:
y_test

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [ ]:
pred = np.float64(res>0.2)
pred

array([[1., 1.],
       [1., 1.],
       [1., 0.],
       [0., 1.],
       [1., 1.],
       [1., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 1.],
       [1., 0.],
       [1., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 1.],
       [1., 0.],
       [1., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 1.],
       [0., 1.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# target = ['class 0', 'class 1']
classification_report(y_test[:,1],pred[:,1])


'              precision    recall  f1-score   support\n\n         0.0       0.53      0.40      0.45       126\n         1.0       0.48      0.61      0.54       114\n\n    accuracy                           0.50       240\n   macro avg       0.51      0.51      0.50       240\nweighted avg       0.51      0.50      0.49       240\n'

In [ ]:
pred = np.int64(res>0.5)
y_label = []
for v in y_test:
  y_label.append(v[0])
y_label=np.array(y_label,dtype=int)

pred_label = []
for v in pred:
  pred_label.append(v[0])
pred_label=np.array(pred_label,dtype=int)
y_label

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1])

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
pred_label

array([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1])

In [ ]:
f1_score(y_label,pred_label)

0.6168831168831169

In [ ]:
print(pred)

[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]

###manual test error data


In [ ]:
# # seed(123)
# # test data
# testErrorSound = readCsv('tinyError')
# print(len(testErrorSound))
# testErrorSoundList = []
# for v in testErrorSound.values():
#     testErrorSoundList.append(np.array(v))
# testErrorSoundList = np.array(testErrorSoundList)
# transposeTestErrorSoundList = testErrorSoundList.transpose((0, 2, 3, 1))

# # transpose of error sound
# print(transposeTestErrorSoundList.shape)

In [ ]:
# testErrorLabels = np.zeros(len(testErrorSound), dtype=int)
# print(testErrorLabels.shape)
# testErrorLabels = to_categorical(testErrorLabels)

In [ ]:
# # test data set
# resTest = model.predict(transposeTestErrorSoundList)
# predTest = np.int64(resTest>0.5)
# test_label = []
# for v in testErrorLabels:
#   test_label.append(v[0])
# test_label=np.array(test_label,dtype=int)

# test_pred_label = []
# for v in predTest:
#   test_pred_label.append(v[0])
# test_pred_label=np.array(test_pred_label,dtype=int)

In [ ]:
# test_score = f1_score(test_label,test_pred_label)
# test_score

In [ ]:
vRange = np.linspace(0.3,0.5,1000)

In [ ]:
# tuple: (threshold,accuracy)
accMap = {"acc":(0.0,0.0)}
for threshold in vRange:
  pred = np.int64(res>threshold)

  # get y_label's index0s
  y_label = []
  for v in y_test:
    y_label.append(v[0])
  y_label=np.array(y_label,dtype=int)

  # get pred_label's index0s
  pred_label = []
  for v in pred:
    pred_label.append(v[0])
  pred_label=np.array(pred_label,dtype=int)
  score = f1_score(y_label,pred_label)
  if score > accMap["acc"][1]:
    accMap["acc"] = (threshold,score)
accMap["acc"]

(0.34184184184184185, 0.9655172413793104)

In [ ]:
print(len(y_label))

240
